## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


The cell below contain the library imports needed to read the CSV data files for The Numbers and Rotten Tomatoes.

In [1]:
#ALIS BRANCH

In [5]:
# Your code here - remember to use markdown cells for comments as well!
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
# Testing to see if we can bring and API in to supplement our data
import requests
import json
url = 'https://api.themoviedb.org/3/movie/{movie_id}?api_key=<<503f12af247746f5fc6113f4fed1f3a3>>&language=en-US'
TMDB = requests.get(url)

In [27]:
type(TMDB)

requests.models.Response

In [28]:
# Attemp to access an API
TMDB.status_code

401

In [13]:
TMDB_data = TMDB.json()

In [20]:
TMDB_data['title']

'Fight Club'

In [2]:
#Unzipping the SQL
from zipfile import ZipFile
# specifying the zip file name
file_name = "./zippedData/im.db.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
im.db                                          2021-12-20 16:31:38    169443328
Extracting all the files now...
Done!


In [3]:
#Establish the SQL connection
conn = sqlite3.connect('im.db')

In [4]:
# FILES WE WILL USE FOR OUR ANALYSIS
TN_DF = pd.read_csv("./zippedData/tn.movie_budgets.csv.gz")
ROT_TOM_INFO_DF = pd.read_csv("./zippedData/rt.movie_info.tsv.gz", sep="\t")
    
# FILES THAT WE ARE NOT GOING TO USE IN OUR ANALYSIS
ROT_TOM_REV_DF = pd.read_csv("./zippedData/rt.reviews.tsv.gz", sep="\t", encoding='windows-1252')
TMDB_DF = pd.read_csv("./zippedData/tmdb.movies.csv.gz")
BOM_DF = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")

These are the tables that have the information we are going to combine.

In [5]:
cur = conn.cursor()

In [6]:
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
# Fetch the result and store it in table_names
table_names = cur.fetchall()
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [7]:
IMDB_ratings = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_ratings;""").fetchall(),
    columns=[x[0] for x in cur.description]
)
IMDB_ratings.shape

(73856, 3)

In [45]:
#This sets the var sql = to the data frame
IMDB = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_basics;""").fetchall(),
    columns=[x[0] for x in cur.description]
)

In [59]:
IMDB['start_year'].min()

2010

In [47]:
IMDB_clean = IMDB.copy()

In [55]:
IMDB_clean = IMDB_clean[IMDB['start_year'] >= 2010]

In [56]:
IMDB_clean.shape

(146144, 6)

In [11]:
#I Looked it uo the world wide numbers are actually the total numbers so Im renaming that colomn
TN_DF

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


Below we are cleaning the data by removing the ID column and also the $ sign and commas from the box office numbers. We also are converting the box offices numbers to int in order to make a new column to represent net income.

In [12]:
TN_clean = TN_DF.copy()

In [13]:
TN_clean.drop('id', axis=1, inplace=True)

In [14]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [15]:
# Stripping $

TN_clean['production_budget'] = TN_clean['production_budget'].map(lambda x: x.strip('$,'))
TN_clean['domestic_gross'] = TN_clean['domestic_gross'].map(lambda x: x.strip('$,'))
TN_clean['worldwide_gross'] = TN_clean['worldwide_gross'].map(lambda x: x.strip('$,'))

In [16]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"425,000,000","760,507,625","2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"410,600,000","241,063,875","1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"350,000,000","42,762,350","149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"330,600,000","459,005,868","1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"317,000,000","620,181,382","1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"7,000",0,0
5778,"Apr 2, 1999",Following,"6,000","48,482","240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"5,000","1,338","1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"1,400",0,0


In [17]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 226.0+ KB


In [18]:
# stripping ',''

TN_clean = TN_clean.apply(lambda x: x.str.replace(',', ''))

In [19]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Dec 18 2009,Avatar,425000000,760507625,2776345279
1,May 20 2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,Jun 7 2019,Dark Phoenix,350000000,42762350,149762350
3,May 1 2015,Avengers: Age of Ultron,330600000,459005868,1403013963
4,Dec 15 2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
5777,Dec 31 2018,Red 11,7000,0,0
5778,Apr 2 1999,Following,6000,48482,240495
5779,Jul 13 2005,Return to the Land of Wonders,5000,1338,1338
5780,Sep 29 2015,A Plague So Pleasant,1400,0,0


In [20]:
TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']] = TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(pd.to_numeric)

In [21]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int64 
 3   domestic_gross     5782 non-null   int64 
 4   worldwide_gross    5782 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 226.0+ KB


In [22]:
TN_clean['Net Income'] = TN_clean['worldwide_gross'] - TN_clean['production_budget']

In [23]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income
0,Dec 18 2009,Avatar,425000000,760507625,2776345279,2351345279
1,May 20 2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,Jun 7 2019,Dark Phoenix,350000000,42762350,149762350,-200237650
3,May 1 2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,Dec 15 2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...
5777,Dec 31 2018,Red 11,7000,0,0,-7000
5778,Apr 2 1999,Following,6000,48482,240495,234495
5779,Jul 13 2005,Return to the Land of Wonders,5000,1338,1338,-3662
5780,Sep 29 2015,A Plague So Pleasant,1400,0,0,-1400


In [24]:
TN_clean['release_date'] = pd.to_datetime(TN_clean['release_date'])

In [25]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   Net Income         5782 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 271.2+ KB


In [28]:
TN_clean.sort_values(by=['Net Income'], ascending=False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income
0,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279
42,1997-12-19,Titanic,200000000,659363944,2208208395,2008208395
6,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200
5,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220
33,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864
...,...,...,...,...,...,...
352,2001-04-27,Town & Country,105000000,6712451,10364769,-94635231
341,2019-06-14,Men in Black: International,110000000,3100000,3100000,-106900000
193,2011-03-11,Mars Needs Moms,150000000,21392758,39549758,-110450242
194,2020-12-31,Moonfall,150000000,0,0,-150000000
